# WMS Visualizer
Create a demo for the client

In [46]:
import os
import glob
import csv
import statistics
import random
import json

In [47]:
!pip install flask_cors



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
# import pandas as pd
from classes.Warehouse import Warehouse

## Warehouse Parameters

In [49]:
NUM_BAYS = 6
RACK_LENGTH = 15
CELLS_PER_BAY = 20
CELL_LENGTH = RACK_LENGTH / CELLS_PER_BAY
RACK_WIDTH = 0.8
BAY_WIDTH = RACK_WIDTH / 2
LEVEL_HEIGHT = 0.2
TIME_TAKEN_PER_CELL = 0.8
TIME_ALLOWED = 24

WHR = Warehouse(
    NUM_BAYS,
    RACK_LENGTH,
    CELLS_PER_BAY,
    BAY_WIDTH,
    LEVEL_HEIGHT,
)

WHS = Warehouse(
    NUM_BAYS,
    RACK_LENGTH,
    CELLS_PER_BAY,
    BAY_WIDTH,
    LEVEL_HEIGHT,
)

## WMS Runner

In [50]:
def wms_runner(WH, type):
    print()
    print()
    print()
    print("------------------------- START -------------------------")
    # WH.reset(type="hard")
    # print(WH.bay_iterator.iterator)
    WH.generate_item_list_small("./data/input/order_history.csv")
    
    # create a warehouse configuration (random / smart)
    if type == "random":
        print("Random sprinkle")
        WH.sprinkle_aisle()

    elif type == "smart":
        print("Smart sprinkle")
        WH.smart_sprinkle()


    directory = 'data/orders'

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if not os.path.isfile(file_path): continue

        print("Read orders")
        WH.read_orders(file_path)

        print("Dist order list")
        WH.dist_batched_order_list()

        for worker in WH.workers:
            worker.min_max_order_list(WH.aisle_items)

        total_cells = 0
        total_orders_processed = 0
        for worker in WH.workers:
            total_orders_processed += worker.fulfill_batched_order(WH.time_alloted, WH.time_pressure)
            total_cells += worker.cells_travelled

        WH.orders_processed.append(total_orders_processed)
        WH.costs.append(total_cells)
        WH.reset(type="soft")
        print("--------------- END -----------------")
        print()

## Datagen

In [51]:
def clear_directory(directory_path):
    # Ensure the directory exists
    if not os.path.isdir(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return

    # Use glob to match all files in the directory
    files = glob.glob(os.path.join(directory_path, '*'))
    
    # Loop through and delete each file
    for file in files:
        try:
            os.remove(file)
        except Exception as e:
            print(f"Error deleting file {file}: {e}")
            
def load_order_history(input_file):
    file_path = 'data/order_history_small.csv'
    items = []

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            items.append({
                "id": int(row["id"]),
                "name": row["name"],
                "quantity": int(row["quantity"]),
                "volume": int(row["volume"]),
                "cover": int(row["cover"])
            })

    total_quantity_6m = sum([item['quantity'] for item in items])
    item_probabilities = [item['quantity'] / total_quantity_6m for item in items]

    return items, item_probabilities

# Generate orders
def generate_orders(num_orders, items, item_probabilities):
    orders = []
    total_items = 0
    for _ in range(num_orders):
        num_items_in_order = random.randint(1, 5)  # Each order can have between 1 and 5 items
        order_items = random.choices(items, weights=item_probabilities, k=num_items_in_order)
        orders.append({"items": order_items})
    return orders

def gen_order_list(num_orders, file_num, items, item_probabilities):
    # Generate a list of orders
    orders = generate_orders(num_orders, items, item_probabilities)

    # Save the orders to a JSON file
    with open(f'data/orders/orders{file_num}.json', 'w') as json_file:
        json.dump(orders, json_file, indent=4)

def datagen(num_order_lists, sample_size):
    clear_directory("data/orders/")
    items, item_probabilities = load_order_history("data/order_history_small.csv")
    for file_num in range(num_order_lists):
        gen_order_list(sample_size, file_num, items, item_probabilities)

In [52]:
clear_directory("data/input")

#FLASK


In [53]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import os

In [54]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})


UPLOAD_FOLDER = 'data/input/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Define a global variable to store the configuration type
current_config_type = None

## Upload Order History

In [55]:
@app.route('/upload', methods=['POST'])
def upload():
    global current_config_type
    clear_directory("data/input")
    if 'itemList' not in request.files or 'configType' not in request.form:
        return jsonify({'error': 'No file part or configuration type'})

    file = request.files['itemList']
    current_config_type = request.form['configType']
    timePressure = request.form['timePressure'].lower() == 'true'
    WHS.time_pressure = timePressure
    WHR.time_pressure = timePressure

    filename = file.filename
    if filename == '':
        return jsonify({'error': 'No selected file'})
    
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], "order_history.csv")
    file.save(file_path)

    # on file upload, run the WMS simulation
    WHS.reset(type="hard")
    WHR.reset(type="hard")
    
    wms_runner(WHS, type="smart")
    wms_runner(WHR, type="random")

    return jsonify({
        'filename': file.filename,
    })

## Generate Order List

In [56]:
@app.route('/datagen', methods=['POST'])
def flaskgen():
    data = request.get_json()
    numOrders = int(data.get('numOrders'))
    numOrderFiles = int(data.get('numOrderFiles'))
    datagen(numOrderFiles, numOrders)

    return jsonify({
        'status': 'success'
    })

## Warehouse Outputs

In [57]:
@app.route('/warehouse')
def index():
    req = request.args.get("req")
    global current_config_type

    if req == "items":
        if current_config_type == "random": warehouse_items = WHR.cell_to_item
        elif current_config_type == "smart": warehouse_items = WHS.cell_to_item
        else: return jsonify({'error': 'Configuration type not set'})
        return jsonify(warehouse_items)
    
    elif req == "warehouse":
        if current_config_type == "random": WH = WHR
        elif current_config_type == "smart": WH = WHS
        else: return jsonify({'error': 'Configuration type not set'})

        random_cost = statistics.mean(WHR.costs)
        smart_cost = statistics.mean(WHS.costs)
        
        random_time = (random_cost * TIME_TAKEN_PER_CELL) / 3600
        smart_time = (smart_cost * TIME_TAKEN_PER_CELL) / 3600

        orders_processed = statistics.mean(WH.orders_processed)

        random_orders_processed = statistics.mean(WHR.orders_processed)
        smart_orders_processed = statistics.mean(WHS.orders_processed)

        if current_config_type == "random":
            cost = random_cost
            time = random_time
        else:
            cost = smart_cost
            time = smart_time


        return jsonify({
            'config': current_config_type,
            'cost': cost,
            'distance': cost * CELL_LENGTH,
            'orders_processed': orders_processed,
            'orders_processed_delta': smart_orders_processed - random_orders_processed,
            'time': time,
            'time_delta': random_time - smart_time,
        })

In [45]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2024 10:30:26] "OPTIONS /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:30:26] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:30:47] "OPTIONS /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:30:47] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:31:15] "OPTIONS /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:31:15] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 10:31:22] "POST /upload HTTP/1.1" 200 -





------------------------- START -------------------------
Smart sprinkle



------------------------- START -------------------------
Random sprinkle


127.0.0.1 - - [24/Oct/2024 10:31:22] "GET /warehouse?req=items HTTP/1.1" 200 -
[2024-10-24 10:31:22,745] ERROR in app: Exception on /warehouse [GET]
Traceback (most recent call last):
  File "c:\Users\NikunjManhas\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\NikunjManhas\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\NikunjManhas\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask_cors\extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\NikunjManhas\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", lin




------------------------- START -------------------------
Smart sprinkle



------------------------- START -------------------------
Random sprinkle


...

127.0.0.1 - - [24/Oct/2024 13:26:09] "GET /warehouse?req=warehouse HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:41:26] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:41:26] "GET /warehouse?req=items HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:41:26] "GET /warehouse?req=warehouse HTTP/1.1" 200 -





------------------------- START -------------------------
Smart sprinkle
Read orders
Dist order list
--------------- END -----------------




------------------------- START -------------------------
Random sprinkle
Read orders
Dist order list
--------------- END -----------------



127.0.0.1 - - [24/Oct/2024 13:51:23] "OPTIONS /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:51:23] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:51:24] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:51:26] "POST /datagen HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:52:51] "POST /upload HTTP/1.1" 200 -





------------------------- START -------------------------
Smart sprinkle
Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----------------

Read orders
Dist order list
--------------- END -----

127.0.0.1 - - [24/Oct/2024 13:52:51] "GET /warehouse?req=warehouse HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 13:52:51] "GET /warehouse?req=items HTTP/1.1" 200 -
